In [ ]:
import numpy as np
import sys
from casadi import *

# Import do_mpc package:
import do_mpc
import math 

In [ ]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

In [ ]:
z = model.set_variable(var_type='_x', var_name='z')
theta = model.set_variable(var_type='_x', var_name='theta')
z_dot = model.set_variable(var_type='_x', var_name='z_dot')
theta_dot = model.set_variable(var_type='_x', var_name='theta_dot')

thrust = model.set_variable(var_type='_u', var_name='thrust', shape=(2,1))

Parameters

In [ ]:
x_g = 0
y_g = 0
z_g = 0
x_b = 0
y_b = 0
z_b = -0.01

#added mass coefficients
X_ud = 6.356673886738176
Y_vd = 7.120600295756984
Z_wd = 18.686326861534997
K_pd = 0.185765630747592
M_qd = 0.134823349429660
N_rd = 0.221510466644690
#drag 
Xu = {"Linear": 13.7, "NonLinear": 141}
Yv = {"Linear": 0, "NonLinear": 217}
Zw = {"Linear": 33, "NonLinear": 190}
Kp = {"Linear": 0, "NonLinear": 1.192}
Mq = {"Linear": 0.8, "NonLinear": 0.470}
Nr = {"Linear": 0, "NonLinear": 1.5}

m = 13.5 # Mass of the Robot

I_x, I_y, I_z = 0.26, 0.23, 0.37 # Mass Moments of Inertia

L_h, L_w, L_l = 0.378, 0.575, 0.457 

A_F, A_S, A_T = 0.0877, 0.1131, 0.2049

Volume = 0.0135

Ic = np.array([[0.26, 0, 0],
      [0, 0.23, 0],
      [0, 0, 0.37]])

Ap_F, Ap_S, Ap_T = 0.1727, 0.2174, 0.2628

System dynamics

In [ ]:
# v = [u v w p q r] eta = J v
J = np.array([[cos(theta), 0],
        [0, 1]])
Jinv = inv(J)
Jinv_T = transpose(Jinv)
Jinv @ vertcat(z_dot, theta_dot)

In [ ]:

M_RB = np.array([[m, 0],
              [0, I_y]])
M_A = -1 *np.array([[Z_wd, 0],
              [0, M_qd]])
M = M_RB + M_A

D_lin = np.array([[Zw["Linear"], 0],
              [0, Mq["Linear"]]])

D_nonlin = np.array([[Zw["NonLinear"] * z_dot/cos(theta), 0],
              [0, Mq["NonLinear"] * theta_dot]])
D = D_lin + D_nonlin

W = m * 9.81
B = 1025 * Volume * 9.81

# G = vertcat(
#     -(W - B) * cos(theta),
#     (z_g*W - z_b*B)*sin(theta) + (x_g*W - x_b*B)*cos(theta)
#     )
G =vertcat(
    -(W - B),
    W * z_g * sin(theta)
)

M_star = Jinv_T @ M @ Jinv
D_star = Jinv_T @ D @ Jinv
G_star = Jinv_T @ G
thrust_star = Jinv_T @ thrust



In [ ]:
M_star

In [ ]:
z_ddot = model.set_variable('_z', 'z_ddot')
theta_ddot = model.set_variable('_z', 'theta_ddot')
x_ddot = vertcat(z_ddot, theta_ddot)
x_dot = vertcat(z_dot, theta_dot)

In [ ]:
# x_ddot = inv(M_star) @  (thrust_star - D_star @ x_dot - G_star ) 
# x_ddot[1]

In [ ]:
model.set_rhs('z', z_dot)
model.set_rhs('theta', theta_dot)
model.set_rhs('z_dot', z_ddot)
model.set_rhs('theta_dot', theta_ddot)

In [ ]:
euler_lagrange = M_star @ x_ddot + D_star @ x_dot + G_star - thrust_star
model.set_alg('euler_lagrange', euler_lagrange)

In [ ]:
_x = vertcat(z, theta, z_dot, theta_dot)

In [ ]:
model.set_expression(expr_name='cost', expr=casadi.sum1(_x**2))

In [ ]:
model.setup()

Controller

In [ ]:
mpc = do_mpc.controller.MPC(model)

In [ ]:
setup_mpc = {
    'n_horizon': 10,
    'n_robust': 2,
    'open_loop': 0,
    't_step': 0.04,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 3,
    'collocation_ni': 1,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.linear_solver': 'mumps'}
}
mpc.set_param(**setup_mpc)

In [ ]:
# # cost matrix for tracking the goal point
# Q_goal = np.diag([
#     100, 100,
#     10, 10])
 
# # cost matrix for the action
# Q_weights = np.array([0.1, 0.1])
# Q_u = np.diag(Q_weights) # T, wx, wy, wz
# x_desired = np.array([
#     10, 0, 
#     0, 0
# ])

# x_current = vertcat(z, theta, z_dot, theta_dot)

# Delta_s = x_current - x_desired

In [ ]:
# cost_goal = Delta_s.T @ Q_goal @ Delta_s 
# # cost_u = thrust.T @ Q_u @ thrust

# mterm = cost_goal
# lterm = cost_goal
mterm = model.aux['cost'] # terminal cost
lterm = model.aux['cost'] # terminal cost
 # stage cost

mpc.set_objective(mterm=mterm, lterm=lterm)

# mpc.set_rterm(thrust=1e-2) # input penalty
# mpc.set_rterm(thrust[1]=1e-2) # input penalty

In [ ]:
mterm.shape

In [ ]:
# max_x = np.array([[4.0], [10.0], [4.0], [10.0]])

# # lower bounds of the states
# mpc.bounds['lower','_x','z'] = 0
# mpc.bounds['lower','_x','theta'] = -pi

# # upper bounds of the states
# mpc.bounds['upper','_x','z'] = 20
# mpc.bounds['upper','_x','theta'] = pi
# # lower bounds of the input
# mpc.bounds['lower','_u','thrust'] = -20 * np.ones([2,1])
# # upper bounds of the input
# mpc.bounds['upper','_u','thrust'] =  20 * np.ones([2,1])

In [ ]:
mpc.setup()

In [ ]:
estimator = do_mpc.estimator.StateFeedback(model)

Simulator

In [ ]:
simulator = do_mpc.simulator.Simulator(model)

In [ ]:
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': 0.04
}

simulator.set_param(**params_simulator)

In [ ]:
simulator.setup()

In [ ]:
# Seed
np.random.seed(99)

# Initial state
e = np.ones([model.n_x,1])
x0 = np.random.uniform(-3*e,3*e) # Values between +3 and +3 for all states
mpc.x0 = x0
simulator.x0 = x0
estimator.x0 = x0

# Use initial state to set the initial guess.
mpc.set_initial_guess()

In [ ]:
x0

In [ ]:
u0 = mpc.make_step(x0)

In [ ]:
%%capture
for k in range(50):
    u0 = mpc.make_step(x0)
    print("u0:", u0)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)

Displaying the results

In [ ]:
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18

In [ ]:
import matplotlib.pyplot as plt
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data, figsize=(16,9))
graphics.plot_results()
graphics.reset_axes()
plt.show()